# How to create a submission in codabench

The aim of this notebook is to show how to create a submissionby presenting necessary and optionnal components. This Notebook presents the composition of a submission folder. For submission examples see notebook [7_submission_examples](https://github.com/IRT-SystemX/ml4physim_startingkit_powergrid/blob/main/7_submission_examples.ipynb) and related directory [submission_examples](https://github.com/IRT-SystemX/ml4physim_startingkit_powergrid/tree/main/submission_examples).

The provided submission examples show how to configure a submission for the following configurations:

- Simple submission using pre-implemented functions
- Submission with a custom model
- Submission with a custom scaler
- Submission for a pre-trained model
- Submission for scoring only


**NB**: in the final phase of the competition, submission will always be re-trained to insure fairness.

**NB**: implementation of tensorflow is being prepared, currently the submission process only accept pytorch models.



We provide also some information on the technical environment (see notebook `9_technical environment`).

## Composition
A submission should contain a set of mendatory and optional files depending on the submission type.

**Mendatory files for all submissions**:
- parameters.json (mandatory): file containing all the necessary parameters to configure a submission
- config.ini: configuration file for the augmented simulator (which sets the hyper-parameters and implement the model)

**Optional files**:
- augmented_simulator.py : file containing an implementation of a custom augmented simulator (which implement the model) that will be evaluated
- my_scaler.py : file containing the implementation of a custom scaler
- json_metrics.json : file containing pre-calculated results for which only the scores are evaluated

*There is also the possibility to add a folder for a pre-trained model:*
- trained_model : will contains the pre-trained model saved through an implementation of the "save" function

**NB.** In notebook 7, we provide examples for each of possible configurations.

## Submission
Submission in codabench will be made through the "My Submissions" tab. A zip containing the files associated with the submission is uploaded.
**Be careful to only compress the file and not the folder containing the file**.

![Benchmark table](img/Submission.png)

## Files content

### parameters.json

This file is used to configure the submission. It indicates to the server which type of submission is being run (training and evaluation, evaluation only using a trained model, scoring only) and how to configure it. 

```json
{
  "evaluateonly": false, // whether to only evaluate an already trained model
  "scoringonly": false, // to obtain only score from provided metrics 
  "simulator_config": {
    "simulator_type": "simple_torch", // The simulator type (simple or custom)
    "simulator_file": "simulator_file", // simulator file containing the a custom model
    "name": "Optional_name", // a name for the simulator
    "model_type": "fully_connected", // Module including the simualor class
    "model": "TorchFullyConnected", // The simulator class
    "scaler_type": "simple", // the scaler type (simple or custom)
    "scaler_file": "scaler_file", // a file containing a custom scaler class
    "scaler_module": "standard_scaler", // module containing the scaler class
    "scaler": "StandardScaler", // the scaler class
    "config_name": "DEFAULT", // section name in config.ini file
    "seed": 42,
  },
  "simulator_extra_parameters": {}, // simulator parameters to be set on-the-fly
  "training_config": { // training hyper parameters
    "epochs": 1
  }
}
```

General parameters:

- `evaluateonly` : set to true if training was already done and the trained model is provided in the `trained_model` folder. 
- `scoringonly` : set to true if training and evaluation was already done and a `results.json` file is provided.

simulator_config

This object is used to indicate the configuration of the augmented simulator :
- `simulator_type`: string indicates the type of model being provided : `simple_torch`, `custom_torch` (LIPS model provided in simulator_file), `simple_tf`, `custom` (model provided in simulator_file)
- `simulator_file`: File in which the custom model can be found (at the root of the folder) **without the '.py'**
- `name`: Name of the simulator
- `model`: Model name, either of the implementation in `simulator_file.py`or of an available model in LIPS
- `model_type`: Used for model already implemented in LIPS for loading  
- `scaler_type`: string indicates the type of model being provided : `simple`, `custom`, `None` (note, for a custom simulator implementation the scaler need to be implemented in the model, no scaler is passed)
- `scaler_file`: File in which the custom scaler can be found (at the root of the folder) **without the '.py'**
- `scaler_class`: Used for scaler already implemented in LIPS for loading
- `scaler`: Name of the scaler to be loaded either from LIPS or `scaler_file`
- `config_name`: Name of the configuration to be used from `config.ini`

simulator_extra_parameters
- `simulator_extra_parameters`: An object containing custom input parameters for the model, it will be passed to the model. 

training_config
- `training_config`: An object containing input parameters for the training phase, it will be passed when the `train` function is called. 


### config.ini

Configuration file for the model, see `4_How_to_contribute`, the section defined by `config_name` will be used.

### my_augmented_simulator.py

This file is used to implement a custom model. This model needs to be runnable by the ingestion process. Notebook 7 presents example for reproducing the 2nd example from `4_How_to_contribute`. 
A custom model will need to implement the following function in order to be runnable in the ingestion process :
- __init__(self,benchmark,**kwargs)
- train(self,train_dataset, save_path=None)
- predict(self,dataset,**kwargs)

### my_scaler.py

This file is used to implement a custom scaler. The example provided show how to use it for recreating an already implemented scaler (used in `4_How_to_contribute`). 

### json_metrics.json

Used in combination with `scoringonly`: true, no training or evaluation will be run, the file will be send directly for scoring (only available during the warmup phase)

### trained_model

Used in combination with `evaluateonly`: true, contains a saved trained model (using the save function from the simulator). In this case, training is skipped and the saved model loaded.